# Filtering SpatialData elements with Table Queries

## Introduction

The `spatialdata` framework supports both the representation of `SpatialElement`s (images, labels, points, shapes) and of annotations for these elements. As we explored in the [tables](./tables.ipynb) notebook, some types of `SpatialElement`s can contain annotations within themselves, but the general approach we take is to represent `SpatialElement`s and annotations in separate objects using `AnnData` tables.

In this notebook we introduce **table queries** - a filtering mechanism that allows you to subset both the annotations (tables) and their corresponding spatial elements using expressive query syntax. This functionality is provided by the `filter_table_by_query()` function, which uses the [`annsel`](https://github.com/srivarra/annsel) library for building query expressions. Under the hood, `annsel` uses  [`narwhals`](https://narwhals-dev.github.io/narwhals/), an "*extremely lightweight and extensible compatibility layer between dataframe libraries*". This notebook assumes that you are have familarized yourself with content in the [tables](./tables.ipynb) notebook.

## Setup and Data Loading

Lets start by importing the necessary libraries and loading the example blobs dataset.

In [ ]:
from pathlib import Path

import annsel as an
import numpy as np

import spatialdata as sd
from spatialdata.datasets import blobs

blobs_sdata = blobs()
blobs_sdata

The table in the blobs dataset is rather minimal, so we will artifically add a couple of columns (`cell_type` and `area`) to help illustrate the functionality.

In [ ]:
rng = np.random.default_rng(123456)

blobs_sdata.tables["table"].obs["cell_type"] = rng.choice(
    ["A", "B", "C", "C", "AA", "BB", "CC"], size=blobs_sdata.tables["table"].n_obs
)
blobs_sdata.tables["table"].obs["cell_type_granular"] = rng.choice(
    ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"], size=blobs_sdata.tables["table"].n_obs
)
blobs_sdata.tables["table"].obs["area"] = rng.choice(
    [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], size=blobs_sdata.tables["table"].n_obs
)

## Supported Operations

## Basic Filtering Examples

Now let's explore how to filter our blobs `SpatialData` object using table queries.

The most common use case is to filter based on observations (`obs`):

In [ ]:
blobs_sdata_filtered = sd.filter_by_table_query(blobs_sdata, table_name="table", obs_expr=an.col("cell_type") == "A")
blobs_sdata_filtered

In [ ]:
print(
    f"\nObservations reduced from {blobs_sdata_filtered.tables['table'].n_obs} to {blobs_sdata_filtered.tables['table'].n_obs}"
)

### Breaking Down `an.col("cell_type") == "A"`



**What is `an.col("cell_type")`?**

`an.col("cell_type")` creates a column reference that points to the "cell_type" column (doesn't specify if it's in `obs` or `var`). By assigning this to the `obs_expr` argument, you're telling the function to filter the `obs` component of the AnnData table based on this column. Think of it as saying "I want to work with the cell_type column".


**What does `== "A"` do?**

The equality operator `== "A"` applies a comparison operator to that column reference, creating a boolean condition that will be `True` for rows where cell_type equals "A" and `False` everywhere else.

**Why This Syntax Design?**

These expressions are ran in `narwhals` under the hood to create expressions and run them. If you have a keen eye, you may notice that this syntax is similar to Polars, as the Narwhals API follows as closely as it can to the ergonomics of Polars.


Lets take look at another example, this time we will want to select observations which belong to the `blobs_labels` region.

In [ ]:
blobs_sdata_filtered = sd.filter_by_table_query(
    blobs_sdata,
    table_name="table",
    obs_expr=an.col("region") == "blobs_labels",
)
blobs_sdata_filtered

Since all the observations in the table are from the `blobs_labels` element, The table query will return the same `AnnData` object to SpatialDate. But in terms of the other `SpatilaElements` we can see that it's only kept the `blobss_labels` element.



You can also filter based on numeric values, as you'd expect.

In [ ]:
blobs_sdata_filtered = sd.filter_by_table_query(blobs_sdata, table_name="table", obs_expr=an.col("instance_id") <= 10)
blobs_sdata_filtered

In [ ]:
blobs_sdata_filtered = sd.filter_by_table_query(
    blobs_sdata, table_name="table", obs_expr=an.col("instance_id").is_in([1, 3, 5, 8, 13])
)
blobs_sdata_filtered

## Supported Operators and Expressions

- `an.col("column_name")` - reference a column in `obs` or `var`
  - *Note:* Can be multiple columns, `an.col(["column_name1", "column_name2"])`
- Special "columns":
  - `an.obs_names` - reference observation names (row indices, aka `AnnData.obs_names`)
  - `an.var_names` - reference variable names (column names, aka `AnnData.var_names`)
- Comparison operators:
  - `>`, `>=`, `<`, `<=`, `==`, `!=`
- Membership:
  - `.is_in([list])`
- String methods:
  - `.str.contains()`, `.str.starts_with()`, `.str.ends_with()`
- Logical:
  - `&` (and), `|` (or), `~` (not)

As long as an expression does not perform an aggregation under the hood or change length, it can be passed used.

For a full list of supported operators and expressions, see the corersponding [narwhals documentation](https://narwhals-dev.github.io/narwhals/api-reference/expr/).

We can also combine multiple expressions per table component (`obs`, `var`, etc...)

Here we will select observations that have a cell type which starts with `"A"`, and observations which whose `cell_type_granular` is in `["A", "B", "C"]`.

In [ ]:
blobs_sdata_filtered = sd.filter_by_table_query(
    blobs_sdata,
    table_name="table",
    obs_expr=((an.col("cell_type").str.starts_with("A")) | (an.col("cell_type_granular").is_in(["A", "B", "C"]))),
)
blobs_sdata_filtered

There are two ways to use "and" operators in table queries:

1. Using `&` operator between two expressions
2. Using a tuple of expressions

In [ ]:
blobs_sdata_filtered = sd.filter_by_table_query(
    blobs_sdata,
    table_name="table",
    obs_expr=((an.col("cell_type").str.starts_with("A")), (an.col("cell_type_granular").is_in(["A", "B", "C"]))),
)
blobs_sdata_filtered.tables["table"].obs

In [ ]:
blobs_sdata_filtered = sd.filter_by_table_query(
    blobs_sdata,
    table_name="table",
    obs_expr=((an.col("cell_type").str.starts_with("A")) & (an.col("cell_type_granular").is_in(["A", "B", "C"]))),
)
blobs_sdata_filtered.tables["table"].obs

In [ ]:
blobs_sdata_filtered.tables["table"].var_names

In this example, suppose that the `var_name` `channel_0_sum` is of some importance to you when the expression value for some observation is greater than 125. We can also filter based on that matrix's column.

In [ ]:
blobs_sdata_filtered = sd.filter_by_table_query(
    blobs_sdata,
    table_name="table",
    x_expr=an.col("channel_0_sum") > 125,
)
blobs_sdata_filtered.tables["table"].obs

And of course you can combine different filters across different `AnnData` Table components.

In [ ]:
blobs_sdata_filtered = sd.filter_by_table_query(
    blobs_sdata,
    table_name="table",
    obs_expr=an.col("cell_type") == "B",
    x_expr=an.col("channel_0_sum") > 125,
)
blobs_sdata_filtered.tables["table"].obs

## Using a Real Dataset

To wrap up the notebook, we'll briefly use the queries 

Here we'll take a look querying using the [mibitof dataset](https://spatialdata.scverse.org/en/stable/tutorials/notebooks/datasets/README.html). In addition there is a companion notebook 

In [ ]:
mibitof_zarr_path = Path("~/Downloads/mibitof.zarr").expanduser()

mibitof_sdata = sd.read_zarr(mibitof_zarr_path)
mibitof_sdata

Lets also get a brief look at the `obs` component of the `AnnData` table. Here are a few columns of interest:

- `point`: This is the name of the Field of View (FOV) that an observation belongs to (in this case it's cells)
- `cell_size`: The area of a cell
- `donor`: The donor that the cell is from
- `Cluster`: The cluster / cell type that the cell belongs to
- `batch`: The batch that the cell is from (usually with respect to the donor or point / FOV)
- `library_id`: An identifier pointing to which `SpatialElement` the observation belongs to.

In [ ]:
mibitof_sdata.tables["table"].obs

In this example, we're picking donor "21d7" and keeping `vars` that either start with `"CD"` or are `"ASCT2"` or `"ATP5A"`.

In [ ]:
mibitof_sdata_filtered = sd.filter_by_table_query(
    mibitof_sdata,
    # filter_tables=False,
    table_name="table",
    obs_expr=an.col("donor") == "21d7",
    var_names_expr=(an.var_names.is_in(["ASCT2", "ATP5A"]) | an.var_names.str.starts_with("CD")),
)
mibitof_sdata_filtered

If your spatialdata object has a lot of `SpatialElements` and you only want to apply the filter to a subset of them, you can use the `element_names` parameter to specify which ones you want to use for the filter!

As a final example, let's take it up a few notches and use most of the features of the `filter_by_table_query` function. We will also be using the `method` version of the query instead of the `function`. They behave the same way, except that the `method` version passes in it's own `SpatialData` object.


We'll be subsetting of specific `SpatialElements`, and applying filters across `obs`, `var`, and `x` components of the `AnnData` table with a variety of queries.

In [ ]:
mibitof_sdata_filtered = mibitof_sdata_filtered.filter_by_table_query(
    table_name="table",
    element_names=["point23_labels", "point8_labels"],
    # Filter observations (obs) based on multiple conditions
    obs_expr=(
        # Cells from donor 21d7 OR 90de
        an.col("donor").is_in(["21d7", "90de"])
        # AND cells with size greater than 400
        & (an.col("cell_size") > 400)
        # AND cells that are either Epithelial or contain "Tcell" in their cluster name
        & (an.col("Cluster") == "Epithelial")
        | (an.col("Cluster").str.contains("Tcell"))
    ),
    # Filter variables (var) based on multiple conditions
    var_names_expr=(
        # Select columns that start with CD
        an.var_names.str.starts_with("CD")
        # OR columns that contain "ATP"
        | an.var_names.str.contains("ATP")
        # OR specific columns
        | an.var_names.is_in(["ASCT2", "PKM2", "SMA"])
    ),
    # Filter based on expression values
    x_expr=(
        # Keep cells where ASCT2 is greater than 0.1
        (an.col("ASCT2") > 0.1)
        # AND less than 2 for ASCT2
        & (an.col("ASCT2") < 2)
    ),
    how="right",
)
mibitof_sdata_filtered

To wrap up, there are a few things to note:

1. **NOTE:** `SpatialElements` are filtered, but the components within those elements are not.
   1. For example, when we're filtering by the `obs` table and we get a subset of the Label `SpatialElement`, the individual segmentation masks are not modified, they will have the exact same masks as the original Label `SpatialElement`.
2. A layer of a given `AnnData` table can be used by specifying the `layer` parameter in the `filter_by_table_query` function.
3. You can use either the method or the function, they behave exactly the same.